In [1]:
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import simps
import warnings
from datetime import datetime

%matplotlib inline


In [2]:
warnings.filterwarnings('ignore')

'''1 Task 1 '''

In [3]:
with open('/home/aleksey/AB-Testing-Examples/Karpov Courses/Data/2022-04-01T12_df_sales_detail.csv') as f:
    df = pd.read_csv(f)

In [4]:
df.head(2)

,sale_id,good,price,date,user_id
0,1000001,mexican pizza,720,2022-02-04 10:00:24,1c1543
1,1000002,chefs pizza,840,2022-02-04 10:02:28,a9a6e8


In [8]:
df[['good','price']].groupby(['good']).sum().reset_index().sort_values(by='price',ascending = False).head(3)

,good,price
3,chefs pizza,24558240
6,double pepperoni pizza,22558380
5,chicken bbq pizza,17622540


'''1 Task 3 '''

In [9]:
with open('/home/aleksey/AB-Testing-Examples/Karpov Courses/Data/2022-04-01T12_df_sales.csv') as sales:
    df_sales = pd.read_csv(sales)

with open('/home/aleksey/AB-Testing-Examples/Karpov Courses/Data/2022-04-01T12_df_web_logs.csv') as logs:
    df_logs = pd.read_csv(logs)


In [10]:
df_sales.head(2)

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8


In [11]:
df_logs.head(2)

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5


'''1 Task 7'''

In [19]:
def get_data_subset(df, begin_date, end_date, user_ids=None, columns=None):

    df = df[(df['date'] >= begin_date & df['date']<= end_date) & (df['user_id'].isin(user_ids)) & (df.columns.isin(columns))]

    return df 

In [18]:
df = pd.DataFrame({
    'date': [datetime(2022, 1, 5), datetime(2022, 1, 7)],
    'user_id': ['1', '2'],
})

df.head(2)

,date,user_id
0,2022-01-05,1
1,2022-01-07,2


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     2 non-null      datetime64[ns]
 1   user_id  2 non-null      object        
dtypes: datetime64[ns](1), object(1)
memory usage: 164.0+ bytes


In [23]:
new_df = get_data_subset(df, '2022-01-01', '2022-06-10')

TypeError: 'str' object cannot be interpreted as an integer

'''3 Task 3'''

In [1]:
from mdemodule import get_minimal_determinable_effect

In [27]:
with open('/home/aleksey/AB-Testing-Examples/Karpov Courses/Data/2022-04-01T12_df_sales.csv') as sales:
    df_sales = pd.read_csv(sales)

In [4]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203847 entries, 0 to 203846
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   sale_id      203847 non-null  int64 
 1   date         203847 non-null  object
 2   count_pizza  203847 non-null  int64 
 3   count_drink  203847 non-null  int64 
 4   price        203847 non-null  int64 
 5   user_id      203847 non-null  object
dtypes: int64(4), object(2)
memory usage: 9.3+ MB


In [41]:
'''df_sales['date'] = df_sales['date'].apply(lambda x: str(datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date()))'''


In [29]:
dd_ab = df_sales[(df_sales['date']>='2022-02-21') & (df_sales['date']<='2022-02-28')][['date','user_id','price']]

In [30]:
dd_ab_grouped = dd_ab[['user_id','price']].groupby(['user_id']).sum().reset_index()

In [8]:
dd_ab_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24840 entries, 0 to 24839
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  24840 non-null  object
 1   price    24840 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 388.3+ KB


In [9]:
dd_ab_grouped[dd_ab_grouped['price']>0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24840 entries, 0 to 24839
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  24840 non-null  object
 1   price    24840 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 388.3+ KB


In [80]:
dd_ab_grouped.head(2)

,user_id,price
0,00045f,720
1,0006bb,1260


In [11]:
mean = dd_ab_grouped['price'].mean()
std = dd_ab_grouped['price'].std()

mean,std

(1234.6871980676328, 811.1803387604872)

In [13]:
from mdemodule import get_sample_size_abs

In [14]:
epsilon=20
std=std
alpha=0.05
beta=0.1
round(get_sample_size_abs(epsilon,std,alpha,beta),-1)

34570

In [36]:
from mdemodule import get_minimal_determinable_effect

sample_size=len(dd_ab_grouped) // 2
std=std
alpha=0.05
beta=0.1
get_minimal_determinable_effect(std,sample_size,alpha,beta)

33.36719696756425

In [35]:
from scipy.stats import norm
import math

sample_size=len(dd_ab_grouped) // 2
z_alpha = norm.ppf(1-alpha/2,)
z_beta = norm.ppf(1-beta)
mde = (z_alpha+z_beta)*std/math.sqrt(sample_size)
mde

23.594171244951884

'''3 Task 5 '''